In [11]:
import torch
import time

tests = {
    'devices': [torch.device('cpu')],
    'parallel_envs': [1, 128, 1024, 8192]
}

def test_evaluator(n_times, evaluator, device, parallel_envs):
    timings = []
    for _ in range(n_times):
        completed = torch.zeros(parallel_envs, dtype=torch.bool, device=device, requires_grad=False)
        while not completed.all():
            start_time = time.time()
            _, _, _, _, terminated = evaluator.step()
            tot_time = time.time() - start_time
            timings.append(tot_time)
            completed |= terminated
    
    return parallel_envs / (sum(timings) / len(timings)) 


In [ ]:
from core.algorithms.load import init_evaluator
from envs.load import init_env


if torch.cuda.is_available():
    tests['devices'].append(torch.device('cuda'))

results = dict()
for device in tests['devices']:
    for n_envs in tests['parallel_envs']:
        env = init_env(device, n_envs, {'env_type': 'othello', 'board_size': 8}, False)
        evaluator = init_evaluator({'name': 'random'}, env) 
        result = test_evaluator(10, evaluator, device, n_envs)
        results[(str(device), n_envs)] = result

In [ ]:
import matplotlib.pyplot as plt

plt.bar(range(len(results)), list(results.values()), align='center')
plt.xticks(range(len(results)), list(results.keys()))
plt.ylabel('Steps/Second')
plt.xlabel('Device, # Envs')
plt.title('Othello Env Benchmark')

plt.show()